## Install Tensorflow

In [1]:
#https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb#scrollTo=ynDmeatCWLJK
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 2.4MB/s 


## Check that TPU is being used

In [2]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf
import bert



assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.54.68.178:8470
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 12565642129763162639),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 11551668110416526869),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 16868634285597510936),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 5874152270697172585),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 2274416058428692144),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0

## Clone the BERT files

In [3]:
import sys

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

# import python modules defined by BERT
import modeling
import optimization
import run_classifier
import run_classifier_with_tfhub
import tokenization

# import tfhub 
import tensorflow_hub as hub

Cloning into 'bert_repo'...
remote: Enumerating objects: 336, done.
remote: Total 336 (delta 0), reused 0 (delta 0), pack-reused 336
Receiving objects: 100% (336/336), 283.23 KiB | 3.93 MiB/s, done.
Resolving deltas: 100% (185/185), done.



## Authenticat Google Cloud Storage locations for saving files

In [4]:
!pip install gcsfs

     |████████████████████████████████| 51kB 1.9MB/s 
  Created wheel for gcsfs: filename=gcsfs-0.3.1-py2.py3-none-any.whl size=17936 sha256=d18de33df330ff53a60867fa0dee4090a6fd0488752529dd294a540c6678d033
  Stored in directory: /root/.cache/pip/wheels/9d/2b/6f/86954f0d8caa1173841e62bb780dc0f8693bd268e04a267682
Successfully built gcsfs


In [5]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

FILE_OUTPUT_DIR = 'bert_base_files'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = True #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = True #@param {type:"boolean"}
BUCKET = 'dissertation_bucket' #@param {type:"string"}

if USE_BUCKET:
  FILE_OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, FILE_OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(FILE_OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(FILE_OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(FILE_OUTPUT_DIR))

***** Model output directory: gs://dissertation_bucket/bert_base_files *****


In [6]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'bert_base'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = True #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = True #@param {type:"boolean"}
BUCKET = 'dissertation_bucket' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: gs://dissertation_bucket/bert_base *****


## Select the BERT model to be used (Base or Large)

In [0]:
# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model
BERT_MODEL = 'uncased_L-12_H-768_A-12' #@param {type:"string"}
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'

## Create a tokenizer

In [8]:
tokenizer = run_classifier_with_tfhub.create_tokenizer_from_hub_module(BERT_MODEL_HUB)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


## Import the Train and Eval datasets from GitHub

In [9]:
! wget -cq https://github.com/ScottJK-20190706/tweet_classifier/blob/master/pickle_files//train_data_formatted.pickle?raw=true
! wget -cq https://github.com/ScottJK-20190706/tweet_classifier/blob/master/pickle_files/eval_data_formatted.pickle?raw=true

import pandas as pd

train_data = pd.read_pickle('train_data_formatted.pickle?raw=true')
eval_data = pd.read_pickle('eval_data_formatted.pickle?raw=true')
all_data = train_data.append(eval_data).reset_index()
train_data = train_data.sample(train_data.shape[0]//1)
print(train_data.shape)
print(eval_data.shape)

DATA_COLUMN = 'Tweet'
LABEL_COLUMN = 'class'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

(4083, 16)
(1021, 16)


## Set up the configurations

In [0]:
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
PREDICT_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
MAX_SEQ_LENGTH = 55
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500

label_list = [0, 1]

# Compute number of train and warmup steps from batch size
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_examples = train_data.apply(lambda x: run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)
num_train_steps = int(len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Setup TPU related config
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
NUM_TPU_CORES = 8
ITERATIONS_PER_LOOP = 1000

def get_run_config(output_dir):
  return tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=output_dir,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

## Build model functions

In [11]:
# Force TF Hub writes to the GS bucket we provide.
os.environ['TFHUB_CACHE_DIR'] = OUTPUT_DIR

model_fn = run_classifier_with_tfhub.model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  bert_hub_module_handle=BERT_MODEL_HUB
)

estimator_from_tfhub = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)


INFO:tensorflow:Using config: {'_model_dir': 'gs://dissertation_bucket/bert_base', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.54.68.178:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0840c852e8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.54.68.178:8470', '_evaluation_master': 'grpc://10.54.68.178:8470', '_is_chief': True, '_num_ps_repli

INFO:tensorflow:Using config: {'_model_dir': 'gs://dissertation_bucket/bert_base', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.54.68.178:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0840c852e8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.54.68.178:8470', '_evaluation_master': 'grpc://10.54.68.178:8470', '_is_chief': True, '_num_ps_repli

INFO:tensorflow:_TPUContext: eval_on_tpu True


INFO:tensorflow:_TPUContext: eval_on_tpu True


## Training Function

In [0]:
# Train the model
def model_train(estimator):
  # We'll set sequences to be at most 128 tokens long.
  train_features = run_classifier.convert_examples_to_features(
      train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started training at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(train_examples)))
  print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
  tf.logging.info("  Num steps = %d", num_train_steps)
  train_input_fn = run_classifier.input_fn_builder(
      features=train_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=True,
      drop_remainder=True)
  estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print('***** Finished training at {} *****'.format(datetime.datetime.now()))

## Train the model

In [13]:
model_train(estimator_from_tfhub)

MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...



INFO:tensorflow:Writing example 0 of 4083


INFO:tensorflow:Writing example 0 of 4083


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


...

INFO:tensorflow:Loss for final step: 0.00038948806.


INFO:tensorflow:Loss for final step: 0.00038948806.


INFO:tensorflow:training_loop marked as finished


INFO:tensorflow:training_loop marked as finished


***** Finished training at 2019-11-02 12:01:11.036723 *****


## Function for predicting

In [0]:
import numpy as np

eval_examples = eval_data.apply(lambda x: run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

def model_predict(estimator):
  # Make predictions on a subset of eval examples
  #prediction_examples = processor.get_dev_examples(TASK_DATA_DIR)[:PREDICT_BATCH_SIZE]
  input_features = run_classifier.convert_examples_to_features(eval_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=True)
  predictions = estimator.predict(predict_input_fn)
  
  pred_0 = []
  actual = []

  for example, prediction in zip(eval_examples, predictions):
    print('text_a: %s\ntext_b: %s\nlabel:%s\nprediction:%s\n' % (example.text_a, example.text_b, str(example.label), prediction['probabilities']))
    pred_0 = np.append(pred_0,prediction['probabilities'])
    actual = np.append(actual,example.label)
  
  return(pred_0, actual)

## Make predictions using the Eval Dataset

In [15]:
pred_0, actual = model_predict(estimator_from_tfhub) 

INFO:tensorflow:Writing example 0 of 1021


INFO:tensorflow:Writing example 0 of 1021


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.


INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.


INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.


text_a: the quicker that sturgeon thing is assassinated the bettet the fucked up cunt and she should use scottish £ not…
text_b: None
label:1
prediction:[7.8055676e-04 9.9921942e-01]

text_a: Mrs Sturgeon you are totally 
Removed from reality if you think you know what the Scottish people really want!!
text_b: None
label:0
prediction:[9.9992824e-01 7.1748669e-05]

text_a: It's not that difficult -
text_b: None
label:0
prediction:[9.999082e-01 9.182805e-05]

text_a: I'd slap Sturgeon with a hot iron
text_b: None
label:1
prediction:[7.9877151e-04 9.9920124e-01]

text_a: these 2 second nicola sturgeon
text_b: None
label:0
prediction:[9.9989510e-01 1.0488421e-04]

INFO:tensorflow:prediction_loop marked as finished


INFO:tensorflow:prediction_loop marked as finished


## Get metrics

In [16]:
i=0
pred = []
while i <= len(pred_0)-1:
  if pred_0[i]<=pred_0[i+1]:
    pred = np.append(pred,1)
  else:
    pred = np.append(pred,0)
  i = i+2

from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, auc, roc_curve, accuracy_score #metrics

conf_matrix = confusion_matrix(actual,pred) #build confusion matrix
precision = precision_score(actual,pred) #calculate precision
recall = recall_score(actual,pred) #calculate recall
f1 = f1_score(actual,pred) #calculate f1
fpr, tpr, thresholds = roc_curve(actual,pred)
auc_score = auc(fpr, tpr) #calculate auc
accuracy = accuracy_score(actual,pred) #calculate accuracy
print(conf_matrix)
print('precision = ' + str(precision))
print('recall = ' + str(recall))
print('f1 = ' + str(f1))
print('auc = ' + str(auc_score))
print('accuracy = ' + str(accuracy))


[[821  10]
 [ 21 169]]
precision = 0.9441340782122905
recall = 0.8894736842105263
f1 = 0.9159891598915989
auc = 0.9387199949331813
accuracy = 0.9696376101860921


## Build Metrics Dataframes

In [0]:
metrics = pd.DataFrame({
  'tf' : 'bert',
  'name' : 'bert_base',
  'bp' : 'bert_base',
  'tn' : conf_matrix[0][0],
  'fp' : conf_matrix[0][1],
  'fn' : conf_matrix[1][0],
  'tp' : conf_matrix[1][1],
  'p' : precision,
  'r' : recall,
  'f_1' : f1,
  'auc_sc' : auc_score,
  'acc' : accuracy  
}, index=[0])

prediction_summary = pd.DataFrame({'tweet_id': eval_data['Tweet ID'].values,
                                   'pred': pred,
                                   'model': 'bert_base',
                                   'file': 'eval_data'})

## Save metrics and predictions

In [18]:
metrics.to_csv('gs://dissertation_bucket/bert_base_files/metrics.csv')
prediction_summary.to_pickle('/tmp/prediction_summary.pickle')
!gsutil cp /tmp/prediction_summary.pickle gs://dissertation_bucket/bert_base_files/

Copying file:///tmp/prediction_summary.pickle [Content-Type=application/octet-stream]...
/ [1 files][ 38.6 KiB/ 38.6 KiB]                                                
Operation completed over 1 objects/38.6 KiB.                                     
